# Assignment 4
In the previous workbook, we had a classifier which was designed to picks between two specific digits, one that we called **signal** and the other we called **background**.   

In this assignment, we will want to read in all of the digits, and design a classifier which finds a specific digit (our **signal** again), but **all** of the other 9 digits will serve as the background.   Our background will naturally be 9 times bigger than our signal (unless we limit it).



# Some useful methods
These two methods will be useful for the code blocks we will write.

The "autovivify" function is very useful for making two dimensional counter (actually nested dictionaries).   A sample use follows.


In [0]:
#
# This allows multidimensional counters (and other more complicated strucutres!)
from collections import defaultdict
def autovivify(levels=1, final=dict):
    return (defaultdict(final) if levels < 2 else
            defaultdict(lambda: autovivify(levels-1, final)))
  
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
  '''))
  init_notebook_mode(connected=False)

# Test of autovivify

In [0]:
import random as rd
#
# Define our multi-dimensional counter
digitFoundCounter = autovivify(2,int)
#
# Here is a for loop that runs from 0 to 9 (1 less than the argument of "range")
for i in range(1000):
#
# Generate a random digit
  digit = rd.randint(0, 9)
  digitFoundCounter[digit]['generated'] += 1
#
# For that digit, call it "found" if another random number is <0.8 (so ~80% efficient)
  if rd.random() < 0.8:
    digitFoundCounter[digit]['found'] += 1
    
#
# Now print results
for digit in range(10):
  print("Digit ",digit,"; generated ",digitFoundCounter[digit]['generated'],"; found ",digitFoundCounter[digit]['found'])

# Getting the data:
As usual the data is on github.   If you uncomment the line with **short = "short_"**, the code will run faster.  Once it is working you should use the full sample.

In [0]:
import pandas as pd
#
short = ""
#short = "short_"

#
# Read in all of the other digits
dfAll = pd.DataFrame()
for digit in range(10):
    print("Processing digit ",digit)
    fname = 'https://raw.githubusercontent.com/big-data-analytics-physics/data/master/ch3/digit_' + short + str(digit) + '.csv'
    df = pd.read_csv(fname,header=None)
    df['digit'] = digit
    dfAll = pd.concat([dfAll, df])


# Defining Signal
At the top of this block we define which of the 10 digits we want to use for our **signal**.

In [0]:
#
# Define our "signal" digit
digitSignal = 5

dfA = dfAll[dfAll['digit']==digitSignal]
dfB = dfAll[dfAll['digit']!=digitSignal]


dfA['signal'] = 1
dfB['signal'] = 0

print("Length of signal sample:     ",len(dfA))
print("Length of background sample: ",len(dfB))


# Task1: Shuffle background
You will need to **shuffle** or randomize the rows of the background data.  We already read the digits in, but they were in batches of the the same digit.   So we need to shuffle them to mix the digits up.  To igure out how to do this, google **sklearn shuffle pandas**.   Note that the "shuffle" method from sklearn creates copies of the orginal dataframe.

In [0]:
# Shuffle the data here


# Tasks 2: Limit background
Come up with a method to limit the rows of the background data that you use, so that it is the **same length** (in rows) as the signal dataframe.   You will want some easy way to turn this on and off.   Run first with the background limited to the **same** length as the signal.   Later you can come back and use all of the background data.


In [0]:
# Your code goes here
#


# Task 3: Combining data
After steps 1 and 2 , you will have a signal dataframe, and a randomized background dataframe.   You will need to combine these two into a single dataframe.  We have done this before, but you can look at **pandas concat** function.   Use the name **dfCombined** for the combined dataframe.


In [0]:
# Your code goes here



#  Task 4: runFitter Method
Next you will want to apply an estimator to this dataset. I want you to make a function that does both the test/train split, and then calls the estimator. The function should look like the following "skeleton".   I show the expected inputs and the expected return values.   Note we did all of this in the example workbook.

In [0]:
#
# Here is the skeleton of the method
from sklearn.model_selection import train_test_split
#
# The inputs are:
#     dfCombined: the input dataframe
#     estimator:  this should be an sklearn classifier (only LinearSVC or SGDClassifier are expected to be used)
def runFitter(dfCombined,estimator):
#
# First do a test/train split


#
# Now fit to our training set


#
# Now predict the classes and get the score for our traing set


#
# Now predict the classes and get the score for our test set


#
  return y_train,y_train_pred,y_train_score,y_train_truedigit,y_test,y_test_pred,y_test_score,y_test_truedigit



# Task 5: Run the fitter
Now we can use the function we defined above.   We have to define our estimate (which we get from sklearn) outside of the method, and pass it as an argument to our defined function.   We do it this way because later on we will want to call the method with a different estimator

In [0]:
# Your code goes here
from sklearn.svm import LinearSVC

estimator = LinearSVC(random_state=42) # use dual=False when  n_samples > n_features which is what we have
#estimator = LinearSVC(random_state=42,dual=False,max_iter=5000)    # use dual=False when  n_samples > n_features which is what we have

y_train,y_train_pred,y_train_score,y_train_truedigit,y_test,y_test_pred,y_test_score,y_test_truedigit = runFitter(dfCombined,estimator)

# Task 5: Implement Performance Method
Next, you need to get the performance of the estimator on **both** the training and testing data.   To do this, I want you to make a function to calulate various performance metrics, and return the result.   

In [0]:
#
# Determine the performance
#
# The inputs:
#    y = array of true labels
#    y_pred = array of predicted labels from the **predict** method of the LinearSVC estimator
#    y_score = array of scores from the **decision_function** method of the LinearSVC estimator
#
# The return values:
#    precision,recall,auc are the calculated values of these metrics
#    fpr, tpr, thresholds are lists containing the  "false positive rate", "true positve rate", and "threshold"
#
def binaryPerformance(y,y_pred,y_score):
#
# Assuming a binary classifier with 1=signal, 0=background
  confusionMatrix = autovivify(2,int)
#
# Get entries in confusion matrix, then calculate precision and recall
# The two dimensions of "confusionMatrix" will be confusionMatrix[trueClass][predClass]
  
#
# Get the ROC curve.  We will use the sklearn function to do this
  from sklearn import metrics
  fpr, tpr, thresholds = metrics.roc_curve(y, y_score, pos_label=1)

#
# Get the auc:   
  auc = metrics.roc_auc_score(y, y_score)
  
  return precision,recall,auc,fpr, tpr, thresholds

# Task 6: Call performance method
Call the "binaryPerformance" method for both the training and testing results for your estimator. How do they compare? Look at precision, recall, AUC, and the ROC curve.

In [0]:
#
# Now get the performaance by calling your binaryPerformance method
#   ===> do this separately for test and train data!
# Here is how you do this for the test sample:
precision_test,recall_test,auc_test,fpr_test, tpr_test, thresholds_test = binaryPerformance(y_test,y_test_pred,y_test_score,debug=False)

#
# Now print
print("AUC training data: ",auc_train)
print("AUC testing data:  ",auc_test)


# Plot the ROC!

In [0]:
#import plotly.plotly as py
import numpy as np
from plotly.offline import iplot
import plotly.graph_objs as go

enable_plotly_in_cell()

trace0 = go.Scatter(
    x=fpr_train,
    y=tpr_train,
    text=thresholds_train,
    mode='line',
    name='Trainig set'
)

trace1 = go.Scatter(
    x=fpr_test,
    y=tpr_test,
    text=thresholds_test,
    mode='line',
    name='Testing set'
)

layout = dict(
    title='ROC Curve',
    xaxis=dict(title='FPR'),
    yaxis=dict(title='TPR')
)

data = [trace0,trace1]      #   this is a list because you might want to plot many data sets
iplot(dict(data=data))#,layout=layout))

# Task 7: A different Classifier
Run with a different classifier (and the large background statistics): the SGDClassifier. Compare your results to the LinearSVC classifier.

In [0]:
# Your code goes here

# Get the new estimator
from sklearn.linear_model import SGDClassifier
estimatorSGD = SGDClassifier(random_state=42) 
#estimatorSGD = SGDClassifier(random_state=42,class_weight="balanced") 
#
# Call runFitter



#
# Now get the performance by calling binaryPerformance


print("AUC training data: ",auc_train)
print("AUC testing data:  ",auc_test)
print("AUC SGD training data: ",auc_sgd_train)
print("AUC SGD testing data:  ",auc_sgd_test)


# Plot all 4 ROCs!
There are 2 from the original estimator, and 2 from the new estimator

In [0]:
# Plot all 4 results
import plotly.plotly as py
import numpy as np
from plotly.offline import iplot
import plotly.graph_objs as go

enable_plotly_in_cell()


# Additional tasks:
1.   For a given signal digit, is the accuracy with which background is rejected, dependent upon what the background digit is?   For example, imagine that our signal digit is 4.   Is th accuracy that a 0 is identifed as background the same as the accuracy that a 9 is identified as background?
2.   Limit the signal to 1/10 of its amximum, and the backgorund to the same number.   What is the perfromance of the estimator?   Use LinearSVC for this.